In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

tf.set_random_seed(777)

In [2]:
imdb = keras.datasets.imdb

In [45]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0])

Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [47]:
train_labels[0:5]
# positive or negative review

array([1, 0, 0, 1, 0])

In [5]:
word_index = imdb.get_word_index()

1646592/1641221 [==============================] - 1s 0us/step


In [30]:
# NLP는 전처리가 굉장히 중요

In [7]:
word_index = {k:(v+3) for k,v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [13]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
# ? for things cannot be found
decode_review(train_data[4])

"<START> yet road the time seen of of br describe in as more be influence that beginning other br express mysterious or connected br see much out earth weird br i budget fan budget ship made go ? with he wasn't the time until their roy ? br i better genuinely yet many yet whole yet as br you show two br 4 and cannot time lost more couldn't ? system that all parts don't turn violence ? film where say other is piece al some entertainment eventual time lost to br provides it al as to ? <UNK> ? main is pleased ? it piece off were far simply original show interesting is age together caused week the regret ? town battle like and much little it james br effort film connected that new director go was in ? abandon ? family tv the time seen"

In [29]:
print(train_labels[4])

0


In [31]:
# modeling

In [33]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                      value = word_index["<PAD>"],
                                                      padding = 'post',
                                                      maxlen = 256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                      value = word_index["<PAD>"],
                                                      padding = 'post',
                                                      maxlen = 256)
print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

In [38]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
#relu 완전 연결층
model.add(keras.layers.Dense(16, activation = tf.nn.relu))
model.add(keras.layers.Dense(1, activation = tf.nn.sigmoid))

model.summary()

W0911 00:44:54.666769 4370601408 deprecation.py:506] From /Users/seungmi/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

W0911 01:02:50.624387 4370601408 deprecation.py:323] From /Users/seungmi/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [41]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=40,
                   batch_size=512,
                   validation_data=(x_val, y_val),
                   verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 0s 30us/sample - loss: 0.6921 - acc: 0.5373 - val_loss: 0.6907 - val_acc: 0.5591
Epoch 2/40
15000/15000 [==============================] - 0s 16us/sample - loss: 0.6876 - acc: 0.6797 - val_loss: 0.6840 - val_acc: 0.7254
Epoch 3/40
15000/15000 [==============================] - 0s 17us/sample - loss: 0.6764 - acc: 0.7473 - val_loss: 0.6691 - val_acc: 0.7549
Epoch 4/40
15000/15000 [==============================] - 0s 17us/sample - loss: 0.6546 - acc: 0.7527 - val_loss: 0.6432 - val_acc: 0.7545
Epoch 5/40
15000/15000 [==============================] - 0s 17us/sample - loss: 0.6203 - acc: 0.7883 - val_loss: 0.6065 - val_acc: 0.7823
Epoch 6/40
15000/15000 [==============================] - 0s 17us/sample - loss: 0.5753 - acc: 0.8087 - val_loss: 0.5628 - val_acc: 0.8002
Epoch 7/40
15000/15000 [==============================] - 0s 16us/sample - loss: 0.5246 - acc: 0.8259 - val_loss: 0.

In [43]:
result = model.evaluate(test_data, test_labels)
print(result)

25000/25000 [==============================] - 0s 14us/sample - loss: 0.3380 - acc: 0.8707
[0.33795646896839143, 0.87068]
